# Luenberger State Observer

In [82]:
import numpy as np
import matplotlib.pyplot as plt
import control

Using System from 01

In [83]:
A = np.matrix('0 1; 2 -1')
B = np.matrix("1; 0")
C = np.matrix("1 0")
D = 0
sys = control.ss(A, B, C, D)  # Creating the state space object

First check is the system is observable

In [84]:
# Check system observability
observability_matrix = control.obsv(A, C)
observability_rank = np.linalg.matrix_rank(observability_matrix)
states = A.shape[0]

if observability_rank == states:
    print("The system is observable.")
else:
    print("The system is not observable. Cannot design observer.")

The system is observable.


In [85]:
# Get the fastest pole of System
fast_pole = max(abs(control.poles(sys)))
print(f"Fastest system pole: {-fast_pole}")
observer_poles = [-3*fast_pole, -3*fast_pole-5]  # Multiplying with a negative number since abs from fastest pole give unstable pole
L = control.place(A.T, C.T, observer_poles)  # .T transposes the matrices
print(f"Observer feedback gains, L=[{round(L[0][0],2)}, {round(L[0][1], 2)}]")

Fastest system pole: -2.0
Observer feedback gains, L=[16.0, 52.0]


Creating the Observer

In [99]:
Aobs = A-L.T@C  # @ sign means matrix multiplication in Python
Bobs = np.concatenate((B, L.T), axis=1)  # Merge similar to C = [A B]
print(Bobs)

[[ 1. 16.]
 [ 0. 52.]]
